In [1]:
import os
os.chdir(os.getcwd()[:os.getcwd().rfind('\\')])

In [2]:
import numpy as np
from src.utils.preprocessing_utils import load_image, preprocess_image, get_bounding_boxes, get_corners, get_areas, filter_by_area, get_boxes_sides_length, filter_by_sides
from src.utils.frontend_utils import input_for_frontend
from src.utils.data_utils import get_characters
import cv2
from skimage import io
import os
from tensorflow import keras
import numpy as np
from skimage.transform import resize
from skimage import io, color
from sklearn.preprocessing import LabelEncoder

In [3]:
# %%timeit
image_file = 'data/raw/real_1.jpg'

img = load_image(image_file)
binary_image = preprocess_image(img)
# characteristics
bounding_boxes = get_bounding_boxes(binary_image)
corners = get_corners(bounding_boxes)
areas = get_areas(corners)
widths, heights = get_boxes_sides_length(corners)

# filters
filtered_areas, filtered_corners, area_mask = filter_by_area(areas, corners, filter_value = 0)
filtered_widths, filtered_heights = np.array(widths)[np.array(area_mask)].tolist(), np.array(heights)[np.array(area_mask)].tolist()

filtered_corners, side_mask = filter_by_sides(filtered_corners, filtered_widths, filtered_heights)
filtered_widths, filtered_heights = np.array(filtered_widths)[np.array(side_mask)].tolist(), np.array(filtered_heights)[np.array(side_mask)].tolist()
filtered_areas = np.array(filtered_areas)[np.array(side_mask)].tolist()

filtered_areas, filtered_corners, area_mask = filter_by_area(filtered_areas, filtered_corners, filter_value = np.mean(filtered_areas)/2)
filtered_widths, filtered_heights = np.array(widths)[np.array(area_mask)].tolist(), np.array(heights)[np.array(area_mask)].tolist()


In [4]:
# %%timeit
characters = get_characters(binary_image, filtered_corners)

In [5]:
model_name = 'models/first_cnn/first_cnn.h5'
label_encoder_name = 'models/first_cnn/first_cnn_label_encoder.npy'

In [6]:
model = keras.models.load_model(model_name)
le = LabelEncoder()
le.classes_ = np.load(label_encoder_name, allow_pickle=True)
le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))

In [7]:
import matplotlib.pyplot as plt
def show_image(image, title='', cmap_type='gray'):
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [8]:
from src.utils.data_utils import zero_padding, remove_extra_space_around_characters

In [9]:
GEO_ALPHABET = {'ა': 1, 'ბ': 2, 'გ': 3, 'დ': 4, 'ე': 5, 'ვ': 6, 
                'ზ': 7, 'თ': 8, 'ი': 9, 'კ': 10, 'ლ': 11, 'მ': 12, 
                'ნ': 13, 'ო': 14, 'პ': 15, 'ჟ': 16, 'რ': 17, 'ს': 18, 
                'ტ': 19, 'უ': 20, 'ფ': 21, 'ქ': 22, 'ღ': 23, 'ყ': 24, 
                'შ': 25, 'ჩ': 26, 'ც': 27, 'ძ': 28, 'წ': 29, 'ჭ': 30, 
                'ხ': 31, 'ჯ': 32, 'ჰ': 33} 

In [10]:
predictions = []
for c in characters:
    preprocessed = zero_padding(remove_extra_space_around_characters(c, extra_space_value=0))
    image_cnn = np.zeros(shape=(1, 28, 28))
    image_cnn[0] = zero_padding(remove_extra_space_around_characters(c, extra_space_value=0))
    prediction = model.predict(image_cnn)
    letter, confidence = np.argmax(prediction), np.max(prediction)
    
    predictions.append((le_name_mapping[letter], confidence))

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


In [11]:
input_for_frontend(filtered_corners, predictions, 'models_predictions.json')